# CNN Based Stock Market Prediction

In the following project, 2D CNN has been used to predict the future stock market fluctuations. 

Reference Paper - https://www.sciencedirect.com/science/article/pii/S0957417419301915

### 1. Importing required libraries

In [1]:
import os
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

### 2. Preprocessing

The data is split into training and testing set. Further, the training set is divided into training and validation.

In [2]:
#DATADIR = "./Dataset"
TRAIN_TEST_CUTOFF = '2016-04-21'
TRAIN_VALID_RATIO = 0.75

In [ ]:
data = {}
for filename in os.listdir():
    if not filename.lower().endswith(".csv"):
        continue # read only the CSV files
    #filepath = os.path.join(DATADIR, filename)
    X = pd.read_csv(filename, index_col="Date", parse_dates=True)
    # basic preprocessing: get the name, the classification
    # Save the target variable as a column in dataframe for easier dropna()
    name = X["Name"][0]
    del X["Name"]
    cols = X.columns
    X["Target"] = (X["Close"].pct_change().shift(-1) > 0).astype(int)
    X.dropna(inplace=True)
    # Fit the standard scaler using the training dataset
    index = X.index[X.index < TRAIN_TEST_CUTOFF]
    index = index[:int(len(index) * TRAIN_VALID_RATIO)]
    scaler = StandardScaler().fit(X.loc[index, cols])
    # Save scale transformed dataframe
    X[cols] = scaler.transform(X[cols])
    data[name] = X

In [4]:
data.keys()

dict_keys(['DJI', 'NASDAQ', 'NSE', 'NYA', 'RUT', 'S&P'])

A data generator is formed which can produce batches of data when called by our model.

In [5]:
def datagen(data, seq_len, batch_size, targetcol, kind):
    "As a generator to produce samples for Keras model"
    batch = []
    while True:
        # Pick one dataframe from the pool
        key = random.choice(list(data.keys()))
        df = data[key]
        input_cols = [c for c in df.columns if c != targetcol]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)
        if kind == 'train':
            index = index[:split]   # range for the training set
        elif kind == 'valid':
            index = index[split:]   # range for the validation set
        # Pick one position, then clip a sequence length
        while True:
            t = random.choice(index)      # pick one time step
            n = (df.index == t).argmax()  # find its position in the dataframe
            if n-seq_len+1 < 0:
                continue # can't get enough data for one sequence length
            frame = df.iloc[n-seq_len+1:n+1]
            batch.append([frame[input_cols].values, df.loc[t, targetcol]])
            break
        # if we get enough for a batch, dispatch
        if len(batch) == batch_size:
            X, y = zip(*batch)
            X, y = np.expand_dims(np.array(X), 3), np.array(y)
            yield X, y
            batch = []

### 3. Model Formulation

The model from the paper is implemented. It contains 2 convolution layers with number of filters = 8. The kernel size are (1x82) and (3x1). This is followed by a max pooling layer which reduces the dimension by half. It is followed by another convolution layer of kernel size = (3 x 1) and a Max Pooling layer. Then it is flattened and fed to a Dense layer which returns the output.

In [6]:
def cnnpred_2d(seq_len=60, n_features=82, n_filters=(8,8,8), droprate=0.1):
    "2D-CNNpred model according to the paper"
    model = Sequential([
        Input(shape=(seq_len, n_features, 1)),
        Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
        Conv2D(n_filters[1], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Conv2D(n_filters[2], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Flatten(),
        Dropout(droprate),
        Dense(1, activation="sigmoid")
    ])
    return model

The metrics to evaluate our model are F1 score and accuracy. The mean absolute error is used as the loss function.

In [8]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1macro(y_true, y_pred):
    f_pos = f1_m(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = f1_m(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2

### 4. Model Training

In [9]:
seq_len = 60
batch_size = 128
n_epochs = 20
n_features = 82
 
# Produce CNNpred as a binary classification problem
model = cnnpred_2d(seq_len, n_features)
model.compile(optimizer="adam", loss="mae", metrics=["acc", f1macro])
model.summary()  # print model structure to console
 
# Set up callbacks and fit the model
# We use custom validation score f1macro() and hence monitor for "val_f1macro"
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.h5"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max",
                    verbose=0, save_best_only=True, save_weights_only=False, save_freq="epoch")
]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 1, 8)          664       
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 1, 8)          200       
                                                                 
 max_pooling2d (MaxPooling2  (None, 29, 1, 8)          0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 1, 8)          200       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 1, 8)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 104)               0

In [10]:
model.fit(datagen(data, seq_len, batch_size, "Target", "train"),
          validation_data=datagen(data, seq_len, batch_size, "Target", "valid"),
          epochs=n_epochs, steps_per_epoch=400, validation_steps=10, verbose=1, callbacks=callbacks)

Epoch 1/20
400/400 [==============================] - 78s 192ms/step - loss: 0.4352 - acc: 0.5793 - f1macro: 0.4477 - val_loss: 0.4896 - val_acc: 0.5023 - val_f1macro: 0.4529
Epoch 2/20
  1/400 [..............................] - ETA: 8s - loss: 0.3774 - acc: 0.6797 - f1macro: 0.6763

D:\Learning\Anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 75s 188ms/step - loss: 0.3131 - acc: 0.7109 - f1macro: 0.6973 - val_loss: 0.4930 - val_acc: 0.5063 - val_f1macro: 0.4953
Epoch 3/20
  1/400 [..............................] - ETA: 5s - loss: 0.2573 - acc: 0.7734 - f1macro: 0.7645

D:\Learning\Anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 75s 188ms/step - loss: 0.2636 - acc: 0.7556 - f1macro: 0.7470 - val_loss: 0.5085 - val_acc: 0.4781 - val_f1macro: 0.4501
Epoch 4/20
400/400 [==============================] - 75s 188ms/step - loss: 0.2402 - acc: 0.7759 - f1macro: 0.7685 - val_loss: 0.4625 - val_acc: 0.5352 - val_f1macro: 0.5126
Epoch 5/20
  6/400 [..............................] - ETA: 4s - loss: 0.2261 - acc: 0.7852 - f1macro: 0.7766

...

400/400 [==============================] - 75s 189ms/step - loss: 0.2017 - acc: 0.8076 - f1macro: 0.8019 - val_loss: 0.4615 - val_acc: 0.5406 - val_f1macro: 0.5335
Epoch 9/20
 11/400 [..............................] - ETA: 4s - loss: 0.1950 - acc: 0.8132 - f1macro: 0.8075

D:\Learning\Anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 80s 200ms/step - loss: 0.1940 - acc: 0.8142 - f1macro: 0.8083 - val_loss: 0.4597 - val_acc: 0.5508 - val_f1macro: 0.5477
Epoch 10/20
 12/400 [..............................] - ETA: 3s - loss: 0.1877 - acc: 0.8190 - f1macro: 0.8161

D:\Learning\Anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 76s 190ms/step - loss: 0.1927 - acc: 0.8144 - f1macro: 0.8084 - val_loss: 0.4559 - val_acc: 0.5516 - val_f1macro: 0.5442
Epoch 11/20
400/400 [==============================] - 77s 193ms/step - loss: 0.1841 - acc: 0.8230 - f1macro: 0.8175 - val_loss: 0.4717 - val_acc: 0.5312 - val_f1macro: 0.5269
Epoch 12/20
400/400 [==============================] - 75s 189ms/step - loss: 0.1858 - acc: 0.8213 - f1macro: 0.8157 - val_loss: 0.4814 - val_acc: 0.5133 - val_f1macro: 0.5093
Epoch 13/20
400/400 [==============================] - 82s 206ms/step - loss: 0.1804 - acc: 0.8265 - f1macro: 0.8213 - val_loss: 0.4832 - val_acc: 0.5172 - val_f1macro: 0.5159
Epoch 14/20
400/400 [==============================] - 75s 187ms/step - loss: 0.1814 - acc: 0.8250 - f1macro: 0.8197 - val_loss: 0.4569 - val_acc: 0.5523 - val_f1macro: 0.5474
Epoch 15/20
400/400 [==============================] - 75s 188ms/step - loss: 0.1784 - acc: 0.8276 - f1macro: 0.8224 - val_loss: 0.4

A data generator for the test set is also prepared. The mean absolute error, accuracy and F1 score are measured and reported below.

In [12]:
def testgen(data, seq_len, targetcol):
    "Return array of all test samples"
    batch = []
    for key, df in data.items():
        input_cols = [c for c in df.columns if c != targetcol]
        # find the start of test sample
        t = df.index[df.index >= TRAIN_TEST_CUTOFF][0]
        n = (df.index == t).argmax()
        for i in range(n+1, len(df)+1):
            frame = df.iloc[i-seq_len:i]
            batch.append([frame[input_cols].values, frame[targetcol][-1]])
    X, y = zip(*batch)
    return np.expand_dims(np.array(X),3), np.array(y)

# Prepare test data
test_data, test_target = testgen(data, seq_len, "Target")

# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.5).astype(int)
print("accuracy:", accuracy_score(test_pred, test_target))
print("MAE:", mean_absolute_error(test_pred, test_target))
print("F1:", f1_score(test_pred, test_target))

41/41 [==============================] - 0s 1ms/step
accuracy: 0.5236985236985237
MAE: 0.4763014763014763
F1: 0.41000962463907603


#### In the following section, a modified version of AlexNet has been implemented to improve the model performance.

## 2D CNN (Improved)

### 1. Importing important libraries

In [ ]:
import os
import random

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

### 2. Preprocessing

The data is split into training and testing set. Further, the training set is divided into training and validation.

In [ ]:
#DATADIR = "./Dataset"
TRAIN_TEST_CUTOFF = '2016-04-21'
TRAIN_VALID_RATIO = 0.75

The data is organised into dictionaries with the keys of the dictionary set to the name of the stock market index. Further the data is scaled using 'Standard Scaler'.

In [ ]:
data = {}
for filename in os.listdir():
    if not filename.lower().endswith(".csv"):
        continue # read only the CSV files
    #filepath = os.path.join(DATADIR, filename)
    X = pd.read_csv(filename, index_col="Date", parse_dates=True)
    # basic preprocessing: get the name, the classification
    # Save the target variable as a column in dataframe for easier dropna()
    name = X["Name"][0]
    del X["Name"]
    cols = X.columns
    X["Target"] = (X["Close"].pct_change().shift(-1) > 0).astype(int)
    X.dropna(inplace=True)
    # Fit the standard scaler using the training dataset
    index = X.index[X.index < TRAIN_TEST_CUTOFF]
    index = index[:int(len(index) * TRAIN_VALID_RATIO)]
    scaler = StandardScaler().fit(X.loc[index, cols])
    # Save scale transformed dataframe
    X[cols] = scaler.transform(X[cols])
    data[name] = X

In [ ]:
data.keys()

dict_keys(['DJI', 'NASDAQ', 'S&P', 'NYA', 'RUT'])

A data generator is formed which can produce batches of data when called by our model.

In [ ]:
def datagen(data, seq_len, batch_size, targetcol, kind):
    "As a generator to produce samples for Keras model"
    batch = []
    while True:
        # Pick one dataframe from the pool
        key = random.choice(list(data.keys()))
        df = data[key]
        input_cols = [c for c in df.columns if c != targetcol]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)
        if kind == 'train':
            index = index[:split]   # range for the training set
        elif kind == 'valid':
            index = index[split:]   # range for the validation set
        # Pick one position, then clip a sequence length
        while True:
            t = random.choice(index)      # pick one time step
            n = (df.index == t).argmax()  # find its position in the dataframe
            if n-seq_len+1 < 0:
                continue # can't get enough data for one sequence length
            frame = df.iloc[n-seq_len+1:n+1]
            batch.append([frame[input_cols].values, df.loc[t, targetcol]])
            break
        # if we get enough for a batch, dispatch
        if len(batch) == batch_size:
            X, y = zip(*batch)
            X, y = np.expand_dims(np.array(X), 3), np.array(y)
            yield X, y
            batch = []

### 3. Model Formulation

For my implementation of the AlexNet, I used two convolutional layers of sizes (1x82) and (3x1) with 8 filters. This was followed by a max pooling layer of size (2x1). Then, I followed it with another set of convolutional and max pooling layer, followed by 3 convolutional layers each of size (3x1) and a max pooling layer of size (2x1). This was followed by 3 dense layers of sizes 4096, 4096 and 1. ReLU activation functions was used in all the layers.

In [ ]:
def cnnpred_2d(seq_len=60, n_features=82, n_filters=(8,8,8,8,8,8), droprate=0.1):
    "2D-CNNpred model according to the paper"
    model = Sequential([
        Input(shape=(seq_len, n_features, 1)),
        Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
        Conv2D(n_filters[1], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Conv2D(n_filters[2], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Conv2D(n_filters[3], kernel_size=(3,1), activation="relu"),
        Conv2D(n_filters[4], kernel_size=(3,1), activation="relu"),
        Conv2D(n_filters[5], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Flatten(),
        Dropout(droprate),
        Dense(4096, activation="relu"),
        Dense(4096, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    return model

The metrics to evaluate our model are F1 score and accuracy. The mean absolute error is used as the loss function.

In [ ]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1macro(y_true, y_pred):
    f_pos = f1_m(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = f1_m(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2

### 4. Model Training

In [ ]:
seq_len = 60
batch_size = 128
n_epochs = 20
n_features = 82
 
# Produce CNNpred as a binary classification problem
model = cnnpred_2d(seq_len, n_features)
model.compile(optimizer="adam", loss="mae", metrics=["acc", f1macro])
model.summary()  # print model structure to console
 
# Set up callbacks and fit the model
# We use custom validation score f1macro() and hence monitor for "val_f1macro"
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.h5"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max",
                    verbose=0, save_best_only=True, save_weights_only=False, save_freq="epoch")
]
model.fit(datagen(data, seq_len, batch_size, "Target", "train"),
          validation_data=datagen(data, seq_len, batch_size, "Target", "valid"),
          epochs=n_epochs, steps_per_epoch=400, validation_steps=10, verbose=1, callbacks=callbacks)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 1, 8)          664       
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 1, 8)          200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 1, 8)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 1, 8)          200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 1, 8)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 1, 8)          2

A data generator for the test set is also prepared. The mean absolute error, accuracy and F1 score are measured and reported below.

### 5. Model Testing

In [ ]:
def testgen(data, seq_len, targetcol):
    "Return array of all test samples"
    batch = []
    for key, df in data.items():
        input_cols = [c for c in df.columns if c != targetcol]
        # find the start of test sample
        t = df.index[df.index >= TRAIN_TEST_CUTOFF][0]
        n = (df.index == t).argmax()
        for i in range(n+1, len(df)+1):
            frame = df.iloc[i-seq_len:i]
            batch.append([frame[input_cols].values, frame[targetcol][-1]])
    X, y = zip(*batch)
    return np.expand_dims(np.array(X),3), np.array(y)

# Prepare test data
test_data, test_target = testgen(data, seq_len, "Target")

# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.5).astype(int)
print("accuracy:", accuracy_score(test_pred, test_target))
print("MAE:", mean_absolute_error(test_pred, test_target))
print("F1:", f1_score(test_pred, test_target))

accuracy: 0.5375609756097561
MAE: 0.4624390243902439
F1: 0.6992385786802031


## Conclusion

 - Improved the F1 score from 0.410 to 0.699.

## Thank You